In [43]:
import os
import pandas as pd
import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon
import h3
import numpy as np

Test_dataset - это данные географических местоположений для оценки как возможных местоположений postmates.

In [44]:
RES1 = 8
RES2 = 9

In [45]:
# # прочитайте файл, в котором хранятся точки данных.
dp_loc = os.path.join("use_data", "datapoints", "datapoints.xlsx")
df = pd.read_excel(dp_loc, usecols=lambda x: 'Unnamed' not in x)

In [46]:
df.head()

,address,type,district,adminstrativeDistrict,lat,lon
0,"улица Тимура Фрунзе, д. 8/5",kiosks,хамовники,Центральный административный округ,55.733460,37.592085
1,"Комсомольский проспект, д. 23-25",kiosks,хамовники,Центральный административный округ,55.726996,37.582864
2,Комсомольский проспект,kiosks,хамовники,Центральный административный округ,55.720382,37.573841
3,"улица Пречистенка, д. 1/2",kiosks,хамовники,Центральный административный округ,55.743848,37.601243
4,"Комсомольский проспект, д. 31",kiosks,хамовники,Центральный административный округ,55.724766,37.579871


In [47]:
import re
def get_h3_geo(g, res=RES2):
    lng = g.x if isinstance(g, Point) else g.centroid.x     
    lat = g.y if isinstance(g, Point) else g.centroid.y
    h3_addr = h3.geo_to_h3(lat=lat, lng=lng, resolution=res)
    return h3_addr, lat, lng        

In [48]:
# получить h3 для каждой точки данных с разрешением RES2 = 9
def get_h3(row):
    row['h3'] = h3.geo_to_h3(lat=row['lat'], lng=row['lon'], resolution=RES2) 
    return row
df = df.apply(get_h3, axis=1)
df.head()

,address,type,district,adminstrativeDistrict,lat,lon,h3
0,"улица Тимура Фрунзе, д. 8/5",kiosks,хамовники,Центральный административный округ,55.733460,37.592085,8911aa785afffff
1,"Комсомольский проспект, д. 23-25",kiosks,хамовники,Центральный административный округ,55.726996,37.582864,8911aa78513ffff
2,Комсомольский проспект,kiosks,хамовники,Центральный административный округ,55.720382,37.573841,8911aa785d3ffff
3,"улица Пречистенка, д. 1/2",kiosks,хамовники,Центральный административный округ,55.743848,37.601243,8911aa7aa8bffff
4,"Комсомольский проспект, д. 31",kiosks,хамовники,Центральный административный округ,55.724766,37.579871,8911aa7858bffff


In [49]:
#извлеките текущий обучающий набор данных (тот, который имеет разрешение RES2=9)

train_data_9 = pd.read_excel(os.path.join("osm_features", f"training_data_{str(RES2)}.xlsx")).set_index('h3')
final_dataset = train_data_9.loc[:, ['lng', 'lat']]
train_data_9.drop(columns=['lng', 'lat'], axis=1, inplace=True)

In [50]:
test_dataset = pd.DataFrame(data=[], columns=train_data_9.columns.copy())

In [51]:
for idx, row in df.iterrows():
    # получить адрес h3
    h3_addr = row['h3']
    # получить соседние ячейки
    neighbors = list(h3.k_ring(h3_addr, k=1))
    # sum the features at each smaller hexagone (resolution = RES2 = 9)
    
    df_idx = pd.DataFrame()
    for n in neighbors:
        # получить строку в виде фрейма данных
        
        try:
            n_row = train_data_9.loc[[n], :] 
            df_idx = pd.concat([df_idx, n_row], ignore_index=True)
        except KeyError:
            pass
        
    df_idx = df_idx.sum().to_frame().T
    test_dataset = pd.concat([test_dataset, df_idx], ignore_index=True)


In [52]:
assert test_dataset.shape[0] == df.shape[0]

In [53]:
test_dataset.fillna(0, inplace=True)

In [54]:
# добавьте данные о населении и пассажирах метро в тестовый набор данных

pop_metro = pd.read_excel("training_data_y.xlsx").set_index('h3').loc[:, ['TotalPassengers', 'population']]

In [ ]:
t = pd.concat([df, test_dataset], axis=1)
t.head()

In [56]:
def set_h3_to_8(row):
    lat = row['lat']
    lng = row['lon']
    row['h3'] = h3.geo_to_h3(lat=lat, lng=lng, resolution=RES1)
    return row
t = t.apply(set_h3_to_8, axis=1)

In [57]:
pop_metro = pop_metro.reset_index()
print(pop_metro.head())

                h3  TotalPassengers  population
0  881181b041fffff              0.0         739
1  881181b043fffff              0.0        1240
2  881181b047fffff              0.0        1865
3  881181b049fffff              0.0        1026
4  881181b04bfffff              0.0        1437


In [58]:
data = pd.merge(t, pop_metro, on='h3', how='left')
data = data.loc[df.index, :].fillna(0)

In [59]:
# адрес h3 больше не нужен

data.drop('h3', axis=1, inplace=True)

In [60]:
# сохраните тестовый набор данных в файле Excel.
data.to_excel("test_data.xlsx")